<a href="https://colab.research.google.com/github/anitha67/100DaysofMLCode/blob/master/bertmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import spacy
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import tokenizers
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold


In [ ]:
!pip install transformers

In [ ]:
import re
import spacy
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import tokenizers
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test_data/train_folds_new.csv',encoding="latin1" )

In [ ]:
df.head()

,id,user,note,timestamp,is_first_response,case_id,is_qualitative,Label,kfold
0,1518,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>IP is clean</p>,2022-02-17 03:21:27.185682+00,False,145,False,0,0
1,1499,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Get IP Address info from AbuseIPDB</p><p><b...,2022-02-17 03:04:08.189051+00,False,140,True,1,0
2,1490,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Get IP Address info from AbuseIPDB</p><p><b...,2022-02-17 03:03:49.836278+00,False,139,False,0,0
3,436,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Get IP Type</p><p><br></p><p>Inputs:</p><p>...,2022-02-09 06:00:19.88609+00,False,85,False,0,0
4,739,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",<p>Get IP Type</p><p><br></p><p>Inputs:</p><p>...,2022-02-09 06:11:01.352545+00,False,85,False,0,0


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
def regex_sub(text):
    ip_pattern = r'((((([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5]))([^0-9]|$))|(((([0-9A-Fa-f]{1,4}:){7}([0-9A-Fa-f]{1,4}|:))|(([0-9A-Fa-f]{1,4}:){6}(:[0-9A-Fa-f]{1,4}|((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3})|:))|(([0-9A-Fa-f]{1,4}:){5}(((:[0-9A-Fa-f]{1,4}){1,2})|:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3})|:))|(([0-9A-Fa-f]{1,4}:){4}(((:[0-9A-Fa-f]{1,4}){1,3})|((:[0-9A-Fa-f]{1,4})?:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){3}(((:[0-9A-Fa-f]{1,4}){1,4})|((:[0-9A-Fa-f]{1,4}){0,2}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){2}(((:[0-9A-Fa-f]{1,4}){1,5})|((:[0-9A-Fa-f]{1,4}){0,3}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(([0-9A-Fa-f]{1,4}:){1}(((:[0-9A-Fa-f]{1,4}){1,6})|((:[0-9A-Fa-f]{1,4}){0,4}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:))|(:(((:[0-9A-Fa-f]{1,4}){1,7})|((:[0-9A-Fa-f]{1,4}){0,5}:((25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]?\d)){3}))|:)))(%.+)?([^A-Za-z0-9]|$)))'
    yyyy_date_pattern = r'([1-2][0-9]{3}[-\.\/])((([1-9]|1[0-2]|0[1-9])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[^0-9])|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|0[1-9]|1[0-2])[^0-9]))'
    m_d_y_data_pattern = r'[^0-9]((([1-9]|0[1-9]|1[0-2])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|1[0-2]|0[1-9])))[-\.\/]([1-2][0-9]{3}|[0-9]{2})'
    timestamp_pattern_1 = r'([1-2][0-9]{3}[-\.\/])((([1-9]|1[0-2]|0[1-9])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[^0-9])|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|0[1-9]|1[0-2])))[\s]([0-9]|[0-1][0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])[^0-9]'
    timestamp_pattern_2 = r'[^0-9]((([1-9]|0[1-9]|1[0-2])[-\.\/]([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))|(([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])[-\.\/]([1-9]|1[0-2]|0[1-9])))[-\.\/]([1-2][0-9]{3}|[0-9]{2})[\s]([0-9]|[0-1][0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])[^0-9]'
    windows_file_path = r'[A-Za-z]:[\\][^\s]*'
    unix_file_path = r''
    url_pattern = r'http[s]?:[\/]{2}[^\s]+'
    email_pattern = r'[\S]+@[^\s\.]+\.[\S]+'
    html_tag = r'\<[^\>]*\>'
    text = re.sub(html_tag,' ',text)
    text = re.sub(windows_file_path,' filepath ',text)
    text = re.sub(url_pattern,' url ',text)
    text = re.sub(email_pattern,' email ',text)
    text = re.sub(ip_pattern,' ipaddress ',text)
    text = re.sub(timestamp_pattern_1,' timestamp ',text)
    text = re.sub(timestamp_pattern_2,' timestamp ',text)
    text = re.sub(yyyy_date_pattern,' timestamp ',text)
    text = re.sub(m_d_y_data_pattern,' timestamp ',text)
    text = text.lower()
    text = re.sub('[^a-z0-9\s]',' ',text)
#     text = nlp_oper(text)
    return text

In [ ]:
df['note'] = df['note'].apply(lambda x:regex_sub(x))
df.head()

,id,user,note,timestamp,is_first_response,case_id,is_qualitative,Label,kfold
0,1518,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",ip is clean,2022-02-17 03:21:27.185682+00,False,145,False,0,0
1,1499,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get ip address info from abuseipdb inputs...,2022-02-17 03:04:08.189051+00,False,140,True,1,0
2,1490,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get ip address info from abuseipdb inputs...,2022-02-17 03:03:49.836278+00,False,139,False,0,0
3,436,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get ip type inputs ip ipaddress ...,2022-02-09 06:00:19.88609+00,False,85,False,0,0
4,739,"{""id"": 1, ""email"": ""system@localhost"", ""userna...",get ip type inputs ip ipaddress ...,2022-02-09 06:11:01.352545+00,False,85,False,0,0


In [ ]:
class config:
    DEVICE = "cuda"
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 8
    EPOCHS = 5
    BERT_PATH = '/content/gdrive/My Drive/Colab Notebooks/test_data/bertbaseuncased'
    TRAINING_FILE = "/content/gdrive/My Drive/Colab Notebooks/test_data/train_folds_new.csv"
    TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True,truncation=True)

In [ ]:
class BERTDataset:
    def __init__(self, text, target):
        self.text = text
        self.target = target
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        text = str(self.text[item])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation= True,
        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.long),
        }

In [ ]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids,return_dict=False)
        bo = self.bert_drop(o2)
        output = self.out(bo)
        return output

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.long)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [ ]:
def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)

In [ ]:
def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.long)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.argmax(outputs,axis = 1).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset =BERTDataset(
        text=df_train['note'].values, target=df_train['Label'].values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.TRAIN_BATCH_SIZE, num_workers=4
    )

    valid_dataset = BERTDataset(
        text=df_valid['note'].values, target=df_valid['Label'].values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device(config.DEVICE)
    model = BERTBaseUncased()
    model.to(device)
   
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )
    best_accuracy = 0
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score = {accuracy}")
        target_names = ['class 0', 'class 1']
        print(classification_report(targets, outputs, target_names=target_names))
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), "model_{}.bin".format(fold))
            best_accuracy = accuracy

In [ ]:
run(fold = 0)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at /content/gdrive/My Drive/Colab Notebooks/test_data/bertbaseuncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another 

Accuracy Score = 0.5018450184501845
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       135
     class 1       0.50      1.00      0.67       136

    accuracy                           0.50       271
   macro avg       0.25      0.50      0.33       271
weighted avg       0.25      0.50      0.34       271



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 34/34 [00:04<00:00,  7.07it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy Score = 0.5018450184501845
              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       135
     class 1       0.50      1.00      0.67       136

    accuracy                           0.50       271
   macro avg       0.25      0.50      0.33       271
weighted avg       0.25      0.50      0.34       271



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the work

Accuracy Score = 0.5018450184501845
              precision    recall  f1-score   support

     class 0       0.50      0.80      0.62       135
     class 1       0.51      0.21      0.29       136

    accuracy                           0.50       271
   macro avg       0.50      0.50      0.45       271
weighted avg       0.50      0.50      0.45       271



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 34/34 [00:04<00:00,  7.12it/s]


Accuracy Score = 0.5018450184501845
              precision    recall  f1-score   support

     class 0       0.50      0.03      0.06       135
     class 1       0.50      0.97      0.66       136

    accuracy                           0.50       271
   macro avg       0.50      0.50      0.36       271
weighted avg       0.50      0.50      0.36       271



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 34/34 [00:04<00:00,  7.13it/s]

Accuracy Score = 0.4981549815498155
              precision    recall  f1-score   support

     class 0       0.50      0.70      0.58       135
     class 1       0.50      0.30      0.38       136

    accuracy                           0.50       271
   macro avg       0.50      0.50      0.48       271
weighted avg       0.50      0.50      0.48       271

